# Programmatic RHEED Streaming

## Install package

In [ ]:
#!pip install atomicds

## Streaming Client Setup

In [ ]:
from atomicds.streaming.rheed_stream import RHEEDStreamer

To get an API key to run this example, go to Account Management from your profile.

In [ ]:
api_key = "YOUR_API_KEY_HERE"
streamer = RHEEDStreamer(api_key=api_key) # Add verbosity=4 to get full debug output

## Streaming modes:

- Callback (push)

- Generator (pull)

Both modes stream data. In either case, deliver frames at the original capture cadence—maintain a consistent inter-frame delay of approximately 1 / FPS seconds, as determined by the capture FPS.

**NOTE**: Ensure at least 2 seconds of data is represented in each frame chunk sent.

### A) Callback/push mode example

In [ ]:
# 1) Define capture parameters and a simple mock frame source
fps = 120
chunk_size = 240
H, W = 300, 500
seconds_per_chunk = chunk_size / fps  # for simulating real-time cadence

# 3) Initialize the remote stream (rotations_per_min > 0.0 => rotating)
data_id = stream.initialize(
    fps=fps,
    rotations_per_min=15.0,
    chunk_size=chunk_size,
    stream_name="Demo (callback mode)",
    physical_sample="Physical Sample Name"
)

# 4) Produce and push chunks as they become available (e.g., from a camera callback)
num_chunks = 5
for chunk_idx in range(num_chunks):
    frames = np.random.randint(0, 256, size=(chunk_size, H, W), dtype=np.uint8)  # (N,H,W)
    stream.push(data_id, chunk_idx, frames)  # schedules packaging+upload and returns immediately
    time.sleep(seconds_per_chunk)            # simulate real-time capture cadence

# 5) (Optional) brief grace period so in-flight uploads finish in this toy example
time.sleep(1.0)

# 6) Finalize the stream on the server
stream.finalize(data_id)

### B) Generator/pull mode example

In [ ]:
# 1) Pretend we already have all frames in a numpy array
total_frames, H, W = 1200, 300, 500
frames = np.random.randint(0, 256, size=(total_frames, H, W), dtype=np.uint8)

# 2) A simple generator that yields chunk_size frames and sleeps to simulate fps
def frame_chunks(parser, chunk_size=240, fps=120.0):
    seconds_per_chunk = chunk_size / fps
    for i in range(0, len(parser), chunk_size):
        yield parser[i:i + chunk_size]   # (N,H,W) uint8
        time.sleep(seconds_per_chunk)    # simulate real-time capture

# 3) Initialize the remote stream (use rotations_per_min=0.0 if stationary)
data_id = stream.initialize(
    fps=10.0,
    rotations_per_min=0.0,
    chunk_size=20,
    stream_name="Demo (generator mode)",
    physical_sample="Physical Sample Name"
)

# 4) Stream all chunks via run(...). When it returns, uploads are done.
stream.run(data_id, frame_chunks(parser, chunk_size=20, fps=10.0))

# 5) Tell the server we're finished.
stream.finalize(data_id)